In [1]:
import os
import pandas as pd
import numpy as np
import multiprocessing as mp
import boto3
import json

from loader import vizwiz as vw
from loader import images as im
from itertools import repeat
from IPython.core.display import HTML


from tqdm import tqdm
tqdm.pandas()

# %pip install opencv-python-headless

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
#%pip install opencv-python-headless

### Analyzing the train data

In [4]:
vizwiz = vw.VizWiz(annotation_file='annotations/train.json')

loading annotations into memory...
Done (t=0.31s)
creating index...
index created! imgs = 23431, anns = 100575


In [5]:
vizwiz_df = pd.DataFrame.from_dict(vizwiz.dataset['annotations'], orient='columns')
vizwiz_images_df = pd.DataFrame.from_dict(vizwiz.dataset['images'], orient='columns')

vizwiz_df = vizwiz_df.merge(vizwiz_images_df.rename({'id': 'image_id', 'text_detected': 'image_text_detected'}, axis=1), 
                            on='image_id', how='left')

In [6]:
# caption counts for each image
vizwiz_df['captions_count'] = vizwiz_df.groupby(by=['image_id'])['caption'].transform('count')

# word counts for each caption
vizwiz_df['words_count'] = vizwiz_df['caption'].str.split().apply(len)

# https://stackoverflow.com/questions/61299310/pandas-groupby-with-custom-function-to-return-the-column-values-as-an-array

In [ ]:
%%time

# create df with unique image id and file name
cols = ['image_id', 'file_name']
vizwiz_unique = vizwiz_df.groupby(by=cols, as_index=False).first()[cols]

# find dimension for each image
bucket = 'assistive-vision'
train_imgs = 'vizwiz/train'

json_file = 'TextCaps/annotations/TextCaps_0.1_train.json'
s3 = boto3.client("s3")
response = s3.get_object(Bucket=bucket, Key=json_file)['Body']
print(response.read().decode('utf-8'))
json_content = json.loads(response)


#images = im.ImageS3(bucket)

#def dimension(path, f):
#    fpath = ''.join([path, '/', f])
#    img = images.getImage(fpath)
#    return img.shape

#vizwiz_unique['shape'] = vizwiz_unique['file_name'].progress_apply(lambda f: dimension(f))

# parallelize the tasks to available cpu cores
#with mp.Pool(mp.cpu_count()) as pool:
#    vizwiz_unique['shape'] = pool.starmap(dimension, zip(repeat(train_imgs), vizwiz_unique['file_name']))